In [0]:
## Program is written and run in google colab with GPU support

## Written by  Tanvir

## install the gpu version of the tensorflow

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
## detect the GPU
#device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#  raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))

In [6]:

##only use at colab notebook
from google.colab import files
uploaded = files.upload()

TypeError: ignored

In [0]:
data = pd.read_csv('housing.csv')

In [0]:
data.isnull().sum()

In [0]:
data=data[['housing_median_age','total_rooms','population','households','median_income','median_house_value']]

In [0]:
## select the feature and target
y_val = data['median_house_value']
x_data=data.drop('median_house_value',1)


In [0]:
## train test split
from sklearn.model_selection import train_test_split

In [0]:
x_train,x_test,y_train,y_test = train_test_split(x_data,y_val,test_size=0.3)

In [0]:
x_train.shape

In [0]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result
  
  
## another normalizatoon

#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#scaler.fit(x_train)




In [0]:
x_train_norm = normalize(x_train)
#x_train_norm = scaler.transform(x_train)
x_test_norm = normalize(x_test)
#x_test_norm = scaler.transform(x_test)

In [0]:

#x_train_norm=x_train_norm.drop(['longitude','latitude'],1)

In [0]:
#x_test_norm=x_test_norm.drop(['longitude','latitude'],1)
x_train_norm.head()

In [0]:
#x_test_norm
#X_train = pd.DataFrame(data=x_train_norm,columns=x_train.columns,index=x_train.index)
#x_test_norm
#X_train = pd.DataFrame(data=x_test_norm,columns=x_test.columns,index=x_test.index)

In [0]:
### making input place holder for all those feature
age=tf.feature_column.numeric_column('housing_median_age')
rooms=tf.feature_column.numeric_column('total_rooms')
#bedrooms=tf.feature_column.numeric_column('total_bedrooms')
pop=tf.feature_column.numeric_column('population')
households=tf.feature_column.numeric_column('households')
income=tf.feature_column.numeric_column('median_income')

In [0]:
## input column list
feat_col = [age,rooms,pop,households,income]

In [0]:
##now create the input func and feed all the data based on the input column
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train_norm,y=y_train,batch_size=10,shuffle=False)

In [0]:
## now train with the column list that we created 
## we create a Densely connectly nural network
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_col) ##[6,6,6] means three layers and 6 perceptron each
model.train(input_fn=input_func,steps=20000)

In [0]:
model1 = tf.estimator.LinearRegressor(feature_columns=feat_col)
model1.train(input_fn=input_func,steps=20000)

In [0]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test_norm,batch_size=10,shuffle=False)

In [0]:
## make the prediction and compare with the real y_test

predict_data = model1.predict(predict_input_func)

In [0]:
predict_data

In [0]:
predict_data_list = list(predict_data)

In [0]:
predict_data_list

In [0]:
## we gonna calculate the RMSE
final_pred = [] 

for pred in predict_data_list:
  final_pred.append(pred['predictions'])

In [0]:
final_pred

In [0]:
from sklearn.metrics import mean_squared_error as mse
mse(y_test,final_pred)**0.5

In [0]:
data.describe().transpose()

##mean in the medican_house_value 206855.816909 	

In [0]:
# make a another input function
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test_norm,y=y_test,batch_size=10,shuffle=False)

In [0]:
model1.evaluate(input_fn=eval_input_func,steps=20000)